# Playing Minesweeper using Deep Reinforcement Learning
##### Oleg Yurchenko (20130857), Alisher Tortay (20140904)

## Overview
In our project we decided to play minesweeper game using different Deep Reinforcement Learninig models. So far we examined following models:

1) Policy based agent

2) Q learning agent

3) Supervised learning network for size of 5x5

## Game
We wrote our own version of minesweeper. In our game there is no mine at position (0,0). Interface works as follows:
we open a square using open(pos) function. This functions returns state in the form of array(1,16) (or array(1,64) for 8x8), reward, and whether gane is finished. Reward is 1 if agent opens correct field, -1 if opens mine or already opened field, and 10 if agent opens all non-mine fields. When agents opens mine or already opened field game is finished. State is represented as follows: each field has its own number. If field is covered - 0. Otherwise usual number + 1. Thus, every fiels can have value from 0 to 9. 

## 1) Policy based agent
In this model we used two architectures for neural network.
In first one we used feed forward network with one hidden layer with 128 nodes. We used epsilon-greedy exploration strategy. We feed network with whole board (4x4 and 8x8). For 4x4 board with 3 mines after 10,000 training games rate of wins is 2%. We would get same rate with making random moves. Policy loss doesn't decrease. For 8x8 during 20,000 training iterations we agent could win only two games.
In other architecture we added two convolutional layers of size 3 and 5. But for some reasonsperformance doesn't improve. For 8x8 during 10,000 training iterations we agent could win only one game.

In [ ]:
def conv2d(states, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    states = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    states = tf.nn.bias_add(x, b)
    return tf.nn.relu(states)

def maxpool2d(states, k=2):
    return tf.nn.max_pool(states, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='SAME')

def conv_net(states, weights, biases):
    states = tf.reshape(states, shape=[-1, size, size, 1])

    conv1 = conv2d(states, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

def toHot(inputState):
    temp = []
    s = np.array([])
    for i in range(inputState.shape[1]):
        sample = np.zeros(10)
        sample[int(inputState[0,i])] = 1.0
        temp.append(sample)
    return np.asmatrix(np.append(s, temp))

def policy_network(states):
    weights = {
        'wc1': tf.Variable(tf.random_normal([3, 3, 1, state_dim])),
        'wc2': tf.Variable(tf.random_normal([5, 5, state_dim, state_dim])),
        'wd1': tf.Variable(tf.random_normal([2*2*state_dim, 256])),
        'out': tf.Variable(tf.random_normal([256, num_actions]))
    }
    biases = {
        'bc1': tf.Variable(tf.random_normal([state_dim])),
        'bc2': tf.Variable(tf.random_normal([state_dim])),
        'bd1': tf.Variable(tf.random_normal([256])),
        'out': tf.Variable(tf.random_normal([num_actions]))
    }

    return conv_net(states, weights, biases)

## 2) Q learning agent
In this model we changed usual q learning algorith to fit our problem. We are not interested in long-term reward. Instead we are more interested in the immediate reward. Thus we removed future rewards. We also used table with true positions of mines to produce our target Q values. During 10,000 training games on board 4x4 with 3 mines agent managed to win 900 games. Since minesweeper has strong local dependicies we decided to add convolutional layers. However it had no positive effect on performance

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Convolution2D, Flatten, Dense, Input
from keras import backend as K
import keras.callbacks
import numpy as np

class NN:
    def __init__(self, bsize = (4, 4), gamma = 0.99):
        self.model = Sequential()
        '''
        self.model.add(Convolution2D(nb_filter=bsize[0]*bsize[1], nb_row=3, nb_col=3, activation='relu', border_mode='same', input_shape=(1, bsize[0], bsize[1]), init='uniform'))
        self.model.add(
            Convolution2D(nb_filter=bsize[0] * bsize[1], nb_row=3, nb_col=3, activation='relu', border_mode='same',
                          input_shape=(1, bsize[0], bsize[1]), init='uniform'))
        self.model.add(Flatten())
        '''
        self.model.add(Dense(200, activation='relu', init='uniform', input_dim=bsize[0]*bsize[1]*10))
        self.model.add(Dense(200, activation='relu', init='uniform'))
        self.model.add(Dense(100, activation='linear', init='uniform'))
        self.model.add(Dense(bsize[0]*bsize[1], activation='softmax', init='uniform'))
        opt = keras.optimizers.Adam(lr=0.001)
        self.model.compile(loss='mean_squared_error', optimizer=opt)

## 3) Supervised learning network for size of 5x5
In this model we used neural network with 3 hidden layers. It takes board of 1x25 as input and produces scalar output between 0 and 1. Output is, in some sense, probability that central square of 5x5 is free of mines.
We trained the network using true mine positions and deployed it on every bordering square (this way agent cannot open already opened fields). We also changed representation of state: in each field, instead of one number we use vector of length 10 with only one value equal to 1 and others are 0. We should notice that this model scales easily because we always train the same network. After training our model on 10,000 games on board 8x8 with 8 mines, we test its performance on two boards: 8x8 with 8 mines and 10x10 with 10 mines. Winning rates are 25% and 35%, respectively. We can see that even though we increased the board size, winnig rate also increases. This happened because in second case density of mines is lower (10% compared to 12.5%).

In [ ]:
from game import *
from ExperienceBuf import *
from NN import *
import numpy as np

class Agent:
    def __init__(self, load = False):
        self.bsize = (10, 10)
        self.game = Game(10, 10, 10)
        self.nn = NN(self.bsize)
        if load:
            self.nn.load()
        self.buf = ExperienceBuf()
        self.num_episodes = 5000
        self.max_game_moves = 64
        self.observe = 10
        self.e = 0.0
        self.y = 0.99
        self.mini_batch = 64

    def toHot(self, inputState):
        temp = []
        s = np.array([])
        for i in range(25):
            sample = np.zeros(10)
            sample[int(inputState[i])] = 1.0
            temp.append(sample)
        return np.asmatrix(np.append(s, temp))

    def learn(self):
        count = 0

        for i in range(self.num_episodes):
            self.game.reset()
            _, _, d = self.game.open((0, 0))
            while (not d):
                pmines, true_vals = self.choose_possible_mines()

                states = self.make_states(pmines)
                states.shape = (len(pmines), 1, 25)

                pboard = np.zeros(self.bsize)

                for i in range(len(pmines)):
                    tmp_state = states[i].flatten()
                    tmp_state = self.toHot(tmp_state)
                    self.buf.memorize(tmp_state, true_vals[i])
                    h, w = pmines[i]
                    pboard[h, w] = self.nn.predict(tmp_state)

                #print pmines
                #print states
                #print pboard

                act = np.argmax(pboard.flatten())
                a = (act // self.bsize[1], act % self.bsize[1])

                if np.random.rand(1) < self.e:
                    a = (np.random.randint(self.bsize[0]), np.random.randint(self.bsize[1]))


                #print a
                _, _, d = self.game.open(a)

                count += 1

            if count > self.observe:
                x, y, bsize = self.buf.get_batch(self.mini_batch)
                x = np.asarray(x)
                x.shape = (bsize, 250)
                y = np.asarray(y)
                y.shape = (bsize, 1)
                self.nn.train(x, y)

            if count % 50 == 0:
                self.nn.save()


    def choose_possible_mines(self):
        pmines = []
        true_vals = []
        s = self.game.display_board
        for h in range(self.bsize[0]):
            for w in range(self.bsize[1]):
                if s[h, w] == COVERED:
                    flag = False
                    for i in range(-1, 2):
                        h_new = h + i
                        if (h_new >= 0):
                            for j in range(-1, 2):
                                w_new = w + j
                                if (w_new >= 0):
                                    try:
                                        if (s[h_new, w_new] > 0):
                                            flag = True
                                    except:
                                        pass
                                if flag: break
                        if flag: break

                    if flag:
                        pmines.append((h, w))
                        if self.game.board[h, w] == BOMB: true_vals.append(0)
                        else:true_vals.append(1)
        return pmines, true_vals

    def make_states(self, border):
        states = np.zeros((len(border), 5, 5))
        s = self.game.display_board + 1
        for k in range(len(border)):
            h, w = border[k]
            for i in range(0, 5):
                h_new = h + i - 2
                for j in range(0, 5):
                    w_new = w + j - 2
                    if (h_new >= 0 and w_new >= 0):
                        try:
                            states[k, i, j] = s[h_new, w_new]
                        except:
                            pass
        return states


    def play(self):
        for i in range(self.num_episodes):
            self.game.reset()
            _, _, d = self.game.open((0, 0))
            while (not d):
                pmines, true_vals = self.choose_possible_mines()

                states = self.make_states(pmines)
                states.shape = (len(pmines), 1, 25)

                pboard = np.zeros(self.bsize)

                for i in range(len(pmines)):
                    tmp_state = states[i].flatten()
                    tmp_state = self.toHot(tmp_state)
                    self.buf.memorize(tmp_state, true_vals[i])
                    h, w = pmines[i]
                    pboard[h, w] = self.nn.predict(tmp_state)

                #print pmines
                #print states
                #print pboard

                act = np.argmax(pboard.flatten())
                a = (act // self.bsize[1], act % self.bsize[1])
                _, _, d = self.game.open(a)
